In [1]:
## import libraries
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
from itertools import islice
import numpy as np

from skimage.external import tifffile
from skimage.external.tifffile import imread

import matplotlib.pyplot as plt

import ipywidgets as widgets

import random


from scipy.ndimage.morphology import binary_erosion as br
from skimage import morphology as skmor

from scipy import ndimage
from PIL import Image, ImageDraw

import numpy.ma as ma

from skimage.measure import label, regionprops

In [36]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200207_new data'

In [37]:
file=f'{bigDir}\\cellinfo_190203.csv'
data=pd.read_csv(file)

In [38]:
data

,Unnamed: 0,file,original cell name,movie,last frame movie position x,last frame movie position y,ab1 (MCM),ab1 ch,ab2 (heterochromatin),ab2 ch,DAPI ch,Unnamed: 11,category,molecular age,physical age [min],imaging core,cell id for tracking,group
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,20200203_cell_00.ciz,20200203-06-13-63X 16bit,20200203 simcmxy06,647.518,153.039,MCM3,1.0,HP1beta,2.0,3.0,NaN,early G1,0.225,80min,HIC,NaN,siLuc
2,3,20200203_cell_01.ciz,20200203-06-25-63X 16bit,20200203 simcmxy06,1490.165,332.591,MCM3,1.0,HP1beta,2.0,3.0,NaN,early G1,0.414,20min,HIC,NaN,siLuc
3,4,20200203_cell_02.ciz,20200203-06-27-63X 16bit,20200203 simcmxy06,1560.535,380.034,MCM3,1.0,HP1beta,2.0,3.0,NaN,early G1,0.510,20min,HIC,NaN,siLuc
4,5,20200203_cell_03.ciz,20200203-06-29-63X 16bit,20200203 simcmxy06,930.763,391.252,MCM3,1.0,HP1beta,2.0,3.0,NaN,middel G1,0.684,350min,HIC,NaN,siLuc
5,6,20200203_cell_04.ciz,20200203-06-51-63X 16bit,20200203 simcmxy06,1653.035,635.701,MCM3,1.0,HP1beta,2.0,3.0,NaN,middel G1,0.376,190min,HIC,NaN,siLuc
6,7,20200203_cell_05.ciz,20200203-06-57-63X 16bit,20200203 simcmxy06,1900.489,694.360,MCM3,1.0,HP1beta,2.0,3.0,NaN,middel G1,0.328,190min,HIC,NaN,siLuc
7,8,20200203_cell_06.ciz,20200203-06-61-63X 16bit,20200203 simcmxy06,1493.427,762.175,MCM3,1.0,HP1beta,2.0,3.0,NaN,late G1,0.927,310min,HIC,NaN,siLuc
8,9,20200203_cell_07.ciz,20200203-06-65-63X 16bit,20200203 simcmxy06,1153.435,813.284,MCM3,1.0,HP1beta,2.0,3.0,NaN,middel G1,0.166,190min,HIC,NaN,siLuc
9,10,20200203_cell_08.ciz,20200203-06-71-63X 16bit,20200203 simcmxy06,1555.600,886.418,MCM3,1.0,HP1beta,2.0,3.0,NaN,middel G1,0.270,290min,HIC,NaN,siLuc


In [39]:
data=data.drop(columns=["Unnamed: 0"])

In [40]:
data=data.dropna(subset=["file"])

In [41]:
data['path']=bigDir

In [42]:
data.head()

,file,original cell name,movie,last frame movie position x,last frame movie position y,ab1 (MCM),ab1 ch,ab2 (heterochromatin),ab2 ch,DAPI ch,Unnamed: 11,category,molecular age,physical age [min],imaging core,cell id for tracking,group,path
1,20200203_cell_00.ciz,20200203-06-13-63X 16bit,20200203 simcmxy06,647.518,153.039,MCM3,1.0,HP1beta,2.0,3.0,NaN,early G1,0.225,80min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
2,20200203_cell_01.ciz,20200203-06-25-63X 16bit,20200203 simcmxy06,1490.165,332.591,MCM3,1.0,HP1beta,2.0,3.0,NaN,early G1,0.414,20min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
3,20200203_cell_02.ciz,20200203-06-27-63X 16bit,20200203 simcmxy06,1560.535,380.034,MCM3,1.0,HP1beta,2.0,3.0,NaN,early G1,0.510,20min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
4,20200203_cell_03.ciz,20200203-06-29-63X 16bit,20200203 simcmxy06,930.763,391.252,MCM3,1.0,HP1beta,2.0,3.0,NaN,middel G1,0.684,350min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
5,20200203_cell_04.ciz,20200203-06-51-63X 16bit,20200203 simcmxy06,1653.035,635.701,MCM3,1.0,HP1beta,2.0,3.0,NaN,middel G1,0.376,190min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...


In [46]:
data=data.drop(columns=["Unnamed: 11", ])

In [44]:
data.at[1,'file'].split(".")[0]

'20200203_cell_00'

In [47]:
i=0
for i,cell in islice(data.iterrows(),i,None):
    data.at[i,'path']=bigDir+"\\data_tiff\\"+data.at[i,'file'].split(".")[0]+'.tif'

In [48]:
data.head()

,file,original cell name,movie,last frame movie position x,last frame movie position y,ab1 (MCM),ab1 ch,ab2 (heterochromatin),ab2 ch,DAPI ch,category,molecular age,physical age [min],imaging core,cell id for tracking,group,path
1,20200203_cell_00.ciz,20200203-06-13-63X 16bit,20200203 simcmxy06,647.518,153.039,MCM3,1.0,HP1beta,2.0,3.0,early G1,0.225,80min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
2,20200203_cell_01.ciz,20200203-06-25-63X 16bit,20200203 simcmxy06,1490.165,332.591,MCM3,1.0,HP1beta,2.0,3.0,early G1,0.414,20min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
3,20200203_cell_02.ciz,20200203-06-27-63X 16bit,20200203 simcmxy06,1560.535,380.034,MCM3,1.0,HP1beta,2.0,3.0,early G1,0.510,20min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
4,20200203_cell_03.ciz,20200203-06-29-63X 16bit,20200203 simcmxy06,930.763,391.252,MCM3,1.0,HP1beta,2.0,3.0,middel G1,0.684,350min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
5,20200203_cell_04.ciz,20200203-06-51-63X 16bit,20200203 simcmxy06,1653.035,635.701,MCM3,1.0,HP1beta,2.0,3.0,middel G1,0.376,190min,HIC,NaN,siLuc,Z:\CookLab\Liu\20190816_organizedData_MCM_load...


In [50]:
data.at[1,"path"]

'Z:\\CookLab\\Liu\\20190816_organizedData_MCM_loading\\20200207_new data\\data_tiff\\20200203_cell_00.tif'

In [52]:
data.to_csv(f'{bigDir}\\cellinfo_200203_v2.csv')